In [1]:
import pandas as pd
from datetime import timedelta
import json
import warnings
import numpy as np
# Suppress all warnings
warnings.filterwarnings("ignore")

# Your code that generates warnings


In [2]:
Spo_dict = {
        "name": [
            "SPO 07.12 TO 15.12 (1)",
            "SPO 07.12 TO 15.12 (2)",
            "SPO 18.12 TO 31.01",
            "spo 01.03 to 07.03",
            "SPO 14.03 TO 31.03",
            "SPO 01.04 TO 15.04",
        ],
        "start_date": [
            "2022-12-07",
            "2022-12-07",
            "2022-12-18",
            "2023-03-01",
            "2023-03-14",
            "2023-04-01"
        ],
        "end_date": [
            "2022-12-15",
            "2022-12-15",
            "2023-01-31",
            "2023-03-07",
            "2023-03-31",
            "2023-04-15"
        ]
}

In [3]:
def date_between(date1, start_date, end_date):
    # Check if date1 is between start_date and end_date
    result = ((date1 >= start_date) & (date1 <= end_date))
    return result

In [4]:
file_path = r"D:\vscoded\Credit_App\Credit_App\test files\jonathan.xlsx"
statment = pd.read_excel(file_path,sheet_name='statment')
con = pd.read_excel(file_path,sheet_name= "contract")

In [10]:
end_date_np = np.array(Spo_dict['end_date'], dtype='datetime64').reshape(-1,1)
start_date_np = np.array(Spo_dict['start_date'], dtype='datetime64').reshape(-1,1)


result = np.vectorize(date_between)(statment['Res_date'],start_date_np,end_date_np).T

In [97]:
result

array([[False, False,  True, False, False, False],
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [False, False,  True, False, False, False]])

In [59]:
invoices = np.array(list(pd.read_excel(file_path,sheet_name= Spo_dict['name']).values()))

In [152]:
def invoice_optimizer(cell,invoice):
    
    begin_date = invoice['first date'].iloc[0]
    end_date = invoice['second date'].iloc[-1]
    
    arr = cell['Arrival']
    dep = cell['Departure'] - timedelta(1)
    code = cell['Rate code']

    if begin_date > arr:
        arr = begin_date
    if end_date < dep:
        dep = end_date
    
    date_range = con[(arr<=con["second date"]) & (dep>=con["first date"])]
    empty_df = date_range.empty
    
    days = 0 
    price = 0
    
    if not empty_df:
        date_range['first date'].iloc[0] = arr
        date_range['second date'].iloc[-1] = dep
        
        days = np.array((date_range['second date'] - date_range['first date']).dt.days + 1)
        price = np.array(date_range[cell['Rate code']])
        
    return days, price, arr, dep, empty_df

In [170]:
dict_of_rows = np.array(statment.to_dict(orient='records')).reshape(-1,1)

In [171]:
invoice_tuple = np.vectorize(invoice_optimizer)(dict_of_rows,invoices)

In [194]:
def contract_removal(date1,date2,cell,right_spo,empty_df):
    if (right_spo) and not(empty_df):
        arr = cell['Arrival']
        dep = cell['Departure'] - timedelta(1)
        code = cell['Rate code']
        
        con1 = con[(arr<=con["second date"]) & (date1>=con["first date"])]
        con2 = con[(date2<=con["second date"]) & (dep>=con["first date"])]
        
        con1['first date'].iloc[0] = arr
        con1['second date'].iloc[-1] = date1
        
        
        con2['first date'].iloc[0] = date2
        con2['second date'].iloc[-1] = dep
        
        result = pd.concat([con1, con2], axis=0)

        
        empty_df = result.empty
    else:
        arr = cell['Arrival']
        dep = cell['Departure'] - timedelta(1)
        code = cell['Rate code']
        
        date_range = con[(arr<=con["second date"]) & (dep>=con["first date"])]
        date_range['first date'].iloc[0] = arr
        date_range['second date'].iloc[-1] = date1
        return date_range
    return result
    

In [196]:
invoice_tuple[4]

array([[ True,  True,  True, False, False, False],
       [ True,  True,  True, False, False, False],
       [ True,  True,  True, False, False, False],
       [ True,  True,  True, False, False, False]])

In [198]:
np.vectorize(contract_removal)(invoice_tuple[2],invoice_tuple[3],dict_of_rows,result,invoice_tuple[4])

True
True
True
True
False
False
False
True
True
True
False
False
False
True
True
True
False
False
False
True
True
True
False
False
False


array([[  first date second date  Main Rate D.STD.GV  Main Rate D.STD.PV  \
        3 2023-07-15  2023-07-15                95.0               99.75

           Main Rate D.SUP.PV  Main Rate D.SUP.SV  Main Rate FAM- PV  Main Rate D.ECO  \
        3               104.5              118.75              150.1             85.5

           SGL - STD - GV  SGL - STD - PV  ...  DBL - ECO  TPL - STD - GV  \
        3          166.25        174.5625  ...      171.0           266.0

           TPL - SUP - PV  TPL - SUP - SV  TPL - FAM - PV  TPL - ECO  QAD - FAM - PV  \
        3           292.6           332.5          420.28      239.4          540.36

           DBL - SUP - PV +2CH  DBL - SUP - SV +2CH  DBL - FAM - PV+2CH
        3               261.25              296.875              375.25

        [1 rows x 27 columns]                                                           ,
          first date second date  Main Rate D.STD.GV  Main Rate D.STD.PV  \
        3 2023-07-15  2023-09-01     

In [ ]:
def append_and_pad(original_array, new_array):
    if original_array.ndim == 1:
        original_array = [original_array]
    # Find the maximum length between the two arrays
    max_length = max(len(original_array[0]), len(new_array))
    
    # Pad both arrays with zeros to match the maximum length
    original_array_padded = np.pad(original_array, ((0, 0), (0, max_length - len(original_array[0]))), 'constant')
    new_array_padded = np.pad(new_array, (0, max_length - len(new_array)), 'constant')
    
    # Stack the two arrays vertically
    result = np.vstack((original_array_padded, new_array_padded))
    
    return result

In [162]:
def calculate_total_price(statment,Spo_dict):
    end_date_np = np.array(Spo_dict['end_date'], dtype='datetime64').reshape(-1,1)
    start_date_np = np.array(Spo_dict['start_date'], dtype='datetime64').reshape(-1,1)
    
    result = np.vectorize(date_between)(statment['Res_date'],start_date_np,end_date_np).T
    
    invoices = np.array(list(pd.read_excel(file_path,sheet_name= Spo_dict['name']).values()))
    
    dict_of_rows = np.array(statment.to_dict(orient='records'))
    
    x = np.vectorize(invoice_optimizer)(dict_of_rows.reshape(-1,1),invoices)

(array([[0, 0, 0, 4, 4, 4],
       [0, 0, 0, 3, 3, 3],
       [0, 0, 0, 3, 3, 3],
       [0, 0, 0, 5, 5, 5]]), array([[  0,   0,   0, 190, 190, 190],
       [  0,   0,   0, 163, 163, 163],
       [  0,   0,   0, 180, 180, 180],
       [  0,   0,   0, 420, 420, 420]]), array([[Timestamp('2023-07-15 00:00:00'),
        Timestamp('2023-09-01 00:00:00'),
        Timestamp('2023-07-15 00:00:00'),
        Timestamp('2023-07-15 00:00:00'),
        Timestamp('2023-07-15 00:00:00'),
        Timestamp('2023-07-15 00:00:00')],
       [Timestamp('2023-07-04 00:00:00'),
        Timestamp('2023-09-01 00:00:00'),
        Timestamp('2023-07-04 00:00:00'),
        Timestamp('2023-07-04 00:00:00'),
        Timestamp('2023-07-04 00:00:00'),
        Timestamp('2023-07-04 00:00:00')],
       [Timestamp('2023-07-12 00:00:00'),
        Timestamp('2023-09-01 00:00:00'),
        Timestamp('2023-07-12 00:00:00'),
        Timestamp('2023-07-12 00:00:00'),
        Timestamp('2023-07-12 00:00:00'),
        Timesta